In [2]:
import numpy as np
import pandas as pd
import skfuzzy as fuzz
from sklearn.manifold import TSNE
import plotly.graph_objs as go
import plotly.express as px
from IPython.display import display

# Data untuk klasterisasi: mendefinisikan data yang akan digunakan dalam bentuk dictionary
data = {
    "Obat": ["Parasetamol", "Amoksisilin", "Vitamin C", "Ibuprofen", "Metformin", "Losartan", 
             "Amlodipin", "Simvastatin", "Aspirin", "Omeprazole", "Cefadroxil", "Dexamethasone", 
             "Azitromisin", "Ketorolac", "Furosemide"],
    "Stok Awal": [500, 300, 600, 700, 400, 350, 450, 500, 650, 550, 600, 500, 450, 400, 350],
    "Permintaan Puskesmas": [400, 200, 450, 500, 350, 300, 380, 420, 520, 460, 480, 430, 350, 300, 300],
    "Pemakaian": [350, 180, 400, 450, 300, 270, 320, 400, 470, 420, 450, 390, 330, 270, 290],
    "Sisa Stok": [150, 120, 200, 250, 100, 80, 130, 100, 180, 130, 150, 110, 120, 130, 60]
}

# Mengonversi data dictionary menjadi DataFrame
df = pd.DataFrame(data)

print("Data Obat")

display(df)

# Menyiapkan data untuk klasterisasi Fuzzy C-Means
# Menggabungkan data 'Stok Awal', 'Permintaan Puskesmas', 'Pemakaian', dan 'Sisa Stok' ke dalam satu array
data_for_clustering = np.vstack((df['Stok Awal'], df['Permintaan Puskesmas'], df['Pemakaian'], df['Sisa Stok'])).T

# Menerapkan algoritma Fuzzy C-Means untuk klasterisasi dengan 5 klaster
n_clusters = 5
cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
    data_for_clustering.T, n_clusters, 2, error=0.005, maxiter=1000, init=None)

# Menentukan keanggotaan klaster untuk setiap titik data berdasarkan nilai keanggotaan terbesar
cluster_membership = np.argmax(u, axis=0)

# Menghitung koordinat klaster (sumbu X dan Y) menggunakan nilai keanggotaan klaster
cluster_x = u.T @ cntr[:, 0]
cluster_y = u.T @ cntr[:, 1]

# Menambahkan informasi klaster dan koordinat ke DataFrame
df['Cluster'] = cluster_membership
df['Cluster_X'] = cluster_x
df['Cluster_Y'] = cluster_y

# Melakukan reduksi dimensi dengan t-SNE menjadi 2 dimensi untuk visualisasi
tsne = TSNE(n_components=2, perplexity=5, n_iter=300, random_state=0)
tsne_results = tsne.fit_transform(data_for_clustering)

# Menambahkan hasil t-SNE (2D) ke DataFrame
df['TSNE-1'] = tsne_results[:, 0]
df['TSNE-2'] = tsne_results[:, 1]

# Membuat scatter plot 2D menggunakan Plotly untuk visualisasi klaster Fuzzy C-Means
fig_fuzzy = px.scatter(df, x='Cluster_X', y='Cluster_Y', color='Cluster',
                       hover_data=['Obat', 'Pemakaian', 'Sisa Stok'],
                       title="Fuzzy C-Means Clustering in 2D (Cluster Coordinates)")

# Memindahkan skala warna (color bar) ke bawah plot
fig_fuzzy.update_layout(coloraxis_colorbar=dict(
    orientation="h",
    yanchor="bottom",
    y=-0.3,  # Memindahkan color bar ke bawah plot
    xanchor="center",
    x=0.5  # Menempatkan color bar di tengah
))

# Menambahkan centroid ke plot sebagai tanda 'x' hitam besar
for i, center in enumerate(cntr):
    fig_fuzzy.add_trace(go.Scatter(x=[center[0]], y=[center[1]], 
                                   mode='markers', marker=dict(size=10, color='black', symbol='x'),
                                   name=f'Centroid {i}'))

# Menampilkan plot hasil klasterisasi Fuzzy C-Means
print("Hasil TSNE")
fig_fuzzy.show()

# Membuat scatter plot 2D menggunakan t-SNE untuk visualisasi klaster
fig_tsne = px.scatter(df, x='TSNE-1', y='TSNE-2', color='Cluster',
                      hover_data=['Obat', 'Stok Awal', 'Permintaan Puskesmas', 'Pemakaian', 'Sisa Stok'],
                      title="t-SNE Visualization of Clustering")

# Menampilkan plot hasil t-SNE
print("Hasil TSNE")
fig_tsne.show()

# Menampilkan tabel hasil klasterisasi dengan koordinat klaster dan hasil t-SNE
df_results = df[['Obat', 'Stok Awal', 'Permintaan Puskesmas', 'Pemakaian', 'Sisa Stok', 
                 'Cluster', 'Cluster_X', 'Cluster_Y', 'TSNE-1', 'TSNE-2']]

# Menampilkan DataFrame dengan hasil klasterisasi dan t-SNE
print("Hasil Klasterisasi Fuzzy C-Means")
display(df_results)


Data Obat


,Obat,Stok Awal,Permintaan Puskesmas,Pemakaian,Sisa Stok
0,Parasetamol,500,400,350,150
1,Amoksisilin,300,200,180,120
2,Vitamin C,600,450,400,200
3,Ibuprofen,700,500,450,250
4,Metformin,400,350,300,100
5,Losartan,350,300,270,80
6,Amlodipin,450,380,320,130
7,Simvastatin,500,420,400,100
8,Aspirin,650,520,470,180
9,Omeprazole,550,460,420,130


Hasil TSNE


Hasil TSNE


,Obat,Stok Awal,Permintaan Puskesmas,Pemakaian,Sisa Stok,Cluster,Cluster_X,Cluster_Y,TSNE-1,TSNE-2
0,Parasetamol,500,400,350,150,2,495.527886,409.085607,-12.454890,-24.077278
1,Amoksisilin,300,200,180,120,4,396.076809,321.241249,4.478411,61.496666
2,Vitamin C,600,450,400,200,0,585.857654,461.785355,-18.756031,21.313948
3,Ibuprofen,700,500,450,250,1,674.937572,500.636184,-18.427704,35.158989
4,Metformin,400,350,300,100,3,428.970533,350.914920,18.136810,-24.047441
5,Losartan,350,300,270,80,4,352.762147,286.718675,-8.262618,58.721916
6,Amlodipin,450,380,320,130,3,442.343058,362.447086,-4.015825,-34.477112
7,Simvastatin,500,420,400,100,2,502.556696,421.125398,-9.947873,-7.498304
8,Aspirin,650,520,470,180,1,635.834609,483.114209,-34.268803,30.242346
9,Omeprazole,550,460,420,130,0,556.177824,447.003105,-22.509781,4.634017
